In [2]:
import urllib.request
import json
from selenium import webdriver
import time
import pandas as pd

client_id = 'jXOGVT21kjhtYcpzCtuB'
client_secret = '7Vr6Or5co7'


#[CODE 1]
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("Url Request Success")
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("Error for URL : %s" % (url))
        return None

#[CODE 2]
def getNaverSearch(srcText, start, display):    
    base = "https://openapi.naver.com/v1/search/blog"
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    
    url = base + parameters
    responseDecode = getRequestUrl(url)   #[CODE 1]
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

#[CODE 3]
def getPostLink(srcText, post, jsonPostLink):    
    title = post['title']
    link = post['link']
    
    jsonPostLink.append({'search_title':srcText, 'title':title, 'link': link})
    return   


#[CODE 4]
def getReviewData(netflix_id, srcText, review, jsonReviewData):   
    
    if(len(review) >= 3):
        # 세 개 다 있을 때
        review1 = review[0]
        review2 = review[1]
        review3 = review[2]
        jsonReviewData.append({'netflix_id':netflix_id, 'title':srcText, 'review1':review1, 'review2': review2,
                       'review3':review3})
    elif(len(review) == 2):
        # 두 개 있을 때
        review1 = review[0]
        review2 = review[1]
        review3 = ''
        jsonReviewData.append({'netflix_id':netflix_id, 'title':srcText, 'review1':review1, 'review2': review2,
                       'review3':review3})
    elif(len(review) == 1):
        # 한 개 있을 때
        review1 = review[0]
        review2 = ''
        review3 = ''
        jsonReviewData.append({'netflix_id':netflix_id, 'title':srcText, 'review1':review1, 'review2': review2,
                       'review3':review3})
    else:
        # 아예 없을 때
        if(''.join(review) == ''):
            review1 = ''
            review2 = ''
            review3 = ''
            jsonReviewData.append({'netflix_id':netflix_id, 'title':srcText, 'review1':review1, 'review2': review2,
                       'review3':review3}) 
    
    return    

#[CODE 0]
def blog_MAIN():
    # selenium의 webdriver로 크롬 브라우저를 실행한다.
    path = "C:/Users/user/CRWL_김도은/chromedriver_win32/chromedriver.exe"
    driver = webdriver.Chrome(path)
    driver.implicitly_wait(5)
    
    f = open('D:/Python/netflix_basic_data/search_title_1.txt', 'r', encoding='UTF8')
    count = 0
    jsonPostLink = []
    jsonReviewData = []
    jsonReviewTotal = []

    #넷플릭스 title
    for i in f:
        i = i.replace('\n', '')
        netflix = i.split('\t')
        netflix_id = netflix[0]
        netflix_title = netflix[1]
        srcText = '넷플릭스 '+ netflix_title
        print(count , " - " , srcText)
        count+=1
        
        try:
            review = []
            review_total = []
            jsonResponse = getNaverSearch(srcText, 1, 7)  #[CODE 2]
            total = jsonResponse['total']
            print('전체 검색 : %d 건' %total)
            if(total == 0):
                continue

            for post in jsonResponse['items']:
                getPostLink(srcText, post, jsonPostLink)  #[CODE 3] 
                link = post['link']
                driver.get(link)
                try:
                    driver.switch_to.frame('mainFrame')
                    overlays = ".se-component.se-text.se-l-default"
                    contents = driver.find_elements_by_css_selector(overlays)
                    content_list = []
                    for content in contents:
                        content_list.append(content.text)
                    content_str = ''.join(content_list)
                    if(content_str != ''):
                        review.append(content_str)
                except:
                    print("error")
                    time.sleep(1)
                    continue


            getReviewData(netflix_id, srcText, review, jsonReviewData) #[CODE 3] 
            review_total.append(netflix_id)
            review_total.append(srcText)
            for k in review:
                review_total.append(k)
            jsonReviewTotal.append(review_total)

            if (count % 40 == 0):
                root_dir = "D:/Python/netflix_basic_data/"
                articles = pd.DataFrame(jsonReviewData, columns=['netflix_id','title','review1','review2','review3'])
                articles.to_csv( root_dir + "review.csv", encoding='utf-8-sig')

                df = pd.DataFrame(jsonReviewTotal)
                df.to_csv( root_dir + "review_total.csv", header=None, index=None, encoding='utf-8-sig')
        except:
            print("try _ error")
            
            
    root_dir = "D:/Python/netflix_basic_data/"
    articles = pd.DataFrame(jsonReviewData, columns=['netflix_id','title','review1','review2','review3'])
    articles.to_csv( root_dir + "review.csv", encoding='utf-8-sig')

    df = pd.DataFrame(jsonReviewTotal)
    df.to_csv( root_dir + "review_total.csv", header=None, index=None, encoding='utf-8-sig')
    
    driver.close()
    with open('D:/Python/netflix_basic_data/blog_link.json', 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonPostLink,  indent=4, sort_keys=True,  ensure_ascii=False)

        outfile.write(jsonFile)
        
    

if __name__ == '__main__':
    blog_MAIN()


0  -  넷플릭스 발토
Url Request Success
전체 검색 : 5 건
1  -  넷플릭스 백 투 더 퓨처 3
Url Request Success
전체 검색 : 13513 건
2  -  넷플릭스 큐브
Url Request Success
전체 검색 : 5316 건
3  -  넷플릭스 링 밖에서
Url Request Success
전체 검색 : 1072 건
4  -  넷플릭스 9 (나인)
Url Request Success
전체 검색 : 15 건
5  -  넷플릭스 블랙피쉬: 노토리어스 킬러
Url Request Success
전체 검색 : 10 건
6  -  넷플릭스 조강지처클럽
Url Request Success
전체 검색 : 38 건
7  -  넷플릭스 그린치
Url Request Success
전체 검색 : 203 건
8  -  넷플릭스 레이크 하우스
Url Request Success
전체 검색 : 50 건
error
9  -  넷플릭스 레볼루셔너리 로드
Url Request Success
전체 검색 : 201 건
10  -  넷플릭스 알러뷰 맨
Url Request Success
전체 검색 : 35 건
11  -  넷플릭스 터보
Url Request Success
전체 검색 : 555 건
12  -  넷플릭스 아카메가 벤다!
Url Request Success
전체 검색 : 22 건
error
error
error
error
error
13  -  넷플릭스 미녀 삼총사 3
Url Request Success
전체 검색 : 396 건
14  -  넷플릭스 올드 가드
Url Request Success
전체 검색 : 2077 건
15  -  넷플릭스 알바뛰는 마왕님
Url Request Success
전체 검색 : 36 건
16  -  넷플릭스 사쿠라코 씨의 발밑에는 시체가 묻혀 있다
Url Request Success
전체 검색 : 1831 건
17  -  넷플릭스 학교생활
Url Request Success
전체 검색 : 2819 건
erro

138  -  넷플릭스 혹성탈출: 반격의 서막
Url Request Success
전체 검색 : 43 건
error
139  -  넷플릭스 그린 마일
Url Request Success
전체 검색 : 120 건
140  -  넷플릭스 피구의 제왕
Url Request Success
전체 검색 : 13 건
error
141  -  넷플릭스 데블
Url Request Success
전체 검색 : 4213 건
142  -  넷플릭스 엔들리스 러브 (2014)
Url Request Success
전체 검색 : 6 건
143  -  넷플릭스 신비한 동물사전
Url Request Success
전체 검색 : 568 건
144  -  넷플릭스 극장판 도라에몽: 신 진구의 버스 오브 재팬
Url Request Success
전체 검색 : 890 건
145  -  넷플릭스 돈 렛 고
Url Request Success
전체 검색 : 121 건
146  -  넷플릭스 가족의 탄생
Url Request Success
전체 검색 : 62 건
147  -  넷플릭스 인디펜던스 데이
Url Request Success
전체 검색 : 204 건
148  -  넷플릭스 라스트 크리스마스
Url Request Success
전체 검색 : 239 건
149  -  넷플릭스 라이트하우스
Url Request Success
전체 검색 : 219 건
150  -  넷플릭스 작은 아씨들
Url Request Success
전체 검색 : 2639 건
151  -  넷플릭스 극장판 헬로카봇: 달나라를 구해줘!
Url Request Success
전체 검색 : 1199 건
152  -  넷플릭스 스노우 화이트
Url Request Success
전체 검색 : 178 건
153  -  넷플릭스 왓 라이즈 비니스
Url Request Success
전체 검색 : 13 건
154  -  넷플릭스 메리에겐 뭔가 특별한 것이 있다
Url Request Success
전체 검색 : 52 건
155  -  넷플릭스 

Url Request Success
전체 검색 : 981 건
280  -  넷플릭스 노인을 위한 나라는 없다
Url Request Success
전체 검색 : 452 건
281  -  넷플릭스 내니 맥피 2 - 유모와 마법소동
Url Request Success
전체 검색 : 802 건
282  -  넷플릭스 쓰리 데이즈
Url Request Success
전체 검색 : 226 건
283  -  넷플릭스 오블리비언
Url Request Success
전체 검색 : 101 건
284  -  넷플릭스 바다 탐험대 옥토넛: 대산호초 보호작전
Url Request Success
전체 검색 : 149 건
285  -  넷플릭스 마이 크리스마스 인
Url Request Success
전체 검색 : 223 건
286  -  넷플릭스 마더
Url Request Success
전체 검색 : 3062 건
287  -  넷플릭스 흑집사
Url Request Success
전체 검색 : 64 건
288  -  넷플릭스 페이트 제로
Url Request Success
전체 검색 : 44 건
289  -  넷플릭스 운명의 밤: 무한의 검제
Url Request Success
전체 검색 : 357 건
290  -  넷플릭스 나만이 없는 거리
Url Request Success
전체 검색 : 434 건
291  -  넷플릭스 샤를로트
Url Request Success
전체 검색 : 118 건
292  -  넷플릭스 블레이드 앤 소울
Url Request Success
전체 검색 : 224 건
293  -  넷플릭스 Mr.하트
Url Request Success
전체 검색 : 85 건
294  -  넷플릭스 킬라킬
Url Request Success
전체 검색 : 113 건
295  -  넷플릭스 노 게임 노 라이프
Url Request Success
전체 검색 : 45 건
error
296  -  넷플릭스 니세코이
Url Request Success
전체 검색 : 31 건
error
2